### Importing the dependencies

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

### Loading and Preprocessing the data

In [2]:
dataset_path = 'imgs' #relative path of the dataset

In [3]:
os.listdir(dataset_path) #listing the directory items

['test', 'train']

In [4]:
from tqdm import tqdm

In [11]:

#function to loading and preprocessing the images
def load_and_preprocess_data(dataset_path):
  images = []
  labels = []
  class_names = sorted(os.listdir(dataset_path + '/train'))
  for label,class_name in enumerate(tqdm(class_names)):
    class_path = os.path.join(dataset_path,'train',class_name) #path of each class
    for img_file in os.listdir(class_path):
      img_path = os.path.join(class_path,img_file) # path of each image in a class
      img = cv2.imread(img_path) #load the image
      img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB) #convert the image into RGB color format
      img = cv2.resize(img,(128,128)) #resizing the image
      img = img/255.0 #normalize the image
      images.append(img)
      labels.append(label)
  return np.array(images),np.array(labels),class_names

In [12]:
x ,y ,class_names = load_and_preprocess_data(dataset_path) #loading data

100%|██████████| 10/10 [02:20<00:00, 14.08s/it]


In [27]:
#splitting training and testing
x_train , x_test, y_train , y_test = train_test_split(x,y,test_size = 0.2) 

In [ ]:
#one-hot encoding the y values
y_train_categorical = to_categorical(y_train, num_classes = len(class_names))
y_test_categorical = to_categorical(y_test, num_classes = len(class_names))

### Creating and Compiling the Model

In [29]:
#importing dependencies
from keras import Sequential
from keras import layers

In [30]:
model = Sequential(
    [
        #input layer
        layers.Input(shape =(128,128,3)),
        
        #first CNN layer
        layers.Conv2D(32,kernel_size=(3,3),activation='relu'),
        layers.MaxPooling2D(pool_size = (2,2)),
        
        #second CNN layer
        layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
        layers.MaxPooling2D(pool_size = (2,2)),
        
        #third CNN layer
        layers.Conv2D(128,kernel_size=(3,3),activation='relu'),
        layers.MaxPooling2D(pool_size = (2,2)),
        
        #flattening the output of CNN layers
        layers.Flatten(),
        
        #Artificial Neural Network
        layers.Dense(128,activation = 'relu'),
        layers.Dense(64,activation='relu'),
        layers.Dropout(0.8), #using dropout to reduce bias
        
        #output layer [using softmax because its a muli-class classification]
        layers.Dense(len(class_names),activation = 'softmax'),      
    ]
)

In [31]:
#compiling the model
model.compile(optimizer= 'adam', loss = 'categorical_crossentropy' , metrics = ['accuracy'])

In [32]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,313,546 (12.64 MB)

 Trainable params: 3,313,546 (12.64 MB)

 Non-trainable params: 0 (0.00 B)

In [33]:
len(x_train)

17939

### Training the Model

In [34]:
#Fitting the model
history = model.fit(x_train,y_train_categorical,epochs = 25,batch_size=64,validation_data=(x_test,y_test_categorical), verbose=2)

Epoch 1/25
281/281 - 148s - 527ms/step - accuracy: 0.3497 - loss: 1.7404 - val_accuracy: 0.8957 - val_loss: 0.5107
Epoch 2/25
281/281 - 132s - 471ms/step - accuracy: 0.7099 - loss: 0.7700 - val_accuracy: 0.9726 - val_loss: 0.1233
Epoch 3/25
281/281 - 136s - 483ms/step - accuracy: 0.8144 - loss: 0.4771 - val_accuracy: 0.9848 - val_loss: 0.0646
Epoch 4/25
281/281 - 140s - 500ms/step - accuracy: 0.8523 - loss: 0.3650 - val_accuracy: 0.9909 - val_loss: 0.0409
Epoch 5/25
281/281 - 141s - 503ms/step - accuracy: 0.8683 - loss: 0.3278 - val_accuracy: 0.9904 - val_loss: 0.0401
Epoch 6/25
281/281 - 142s - 506ms/step - accuracy: 0.8876 - loss: 0.2724 - val_accuracy: 0.9897 - val_loss: 0.0335
Epoch 7/25
281/281 - 146s - 520ms/step - accuracy: 0.8936 - loss: 0.2579 - val_accuracy: 0.9940 - val_loss: 0.0330
Epoch 8/25
281/281 - 153s - 545ms/step - accuracy: 0.8984 - loss: 0.2352 - val_accuracy: 0.9949 - val_loss: 0.0228
Epoch 9/25
281/281 - 149s - 532ms/step - accuracy: 0.9058 - loss: 0.2172 - val_a

Achieved an accuracy of 94% using 3 CNN layers

### Save the model

In [83]:
model.save('distracted_driver_model.h5')

### Using the model to predict actual test values

In [5]:
import keras
from keras.models import load_model

In [6]:
model = load_model('distracted_driver_model.h5')

In [7]:
final_path = os.path.join(dataset_path,'test')

In [10]:
def generate_predictions(final_list, batch_size=64):
    predictions = []
    num_batches = len(final_list) // batch_size + 1
    
    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = min((batch_idx + 1) * batch_size, len(final_list))
        
        batch_images = []
        batch_labels = []
        
        for idx in range(start_idx, end_idx):
            img_file = final_list[idx]
            img_path = os.path.join(final_path, img_file)
            
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (128, 128))
            img = img / 255.0
            
            batch_images.append(img)
            batch_labels.append(img_file)
        
        batch_images = np.array(batch_images)
        
        # Predict batch of images
        batch_predictions = model.predict(batch_images)
        
        for i in range(len(batch_labels)):
            predictions.append((batch_labels[i], batch_predictions[i]))
    
    return predictions



In [11]:
final_list = os.listdir(final_path)

In [12]:
predictions = generate_predictions(final_list)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 100ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
2/2 ━━━━━━━

In [25]:
class_names = os.listdir(dataset_path + '/train')
type(class_names)

list

In [38]:
type(predictions[1][0])

str

In [66]:
final_predictions = []
header = ['img']

#giving header
for i in class_names:
    header.append(i)
final_predictions.append((header))  

#appending the predictions
for prediction in tqdm(predictions):
    row = [prediction[0]]
    for i in prediction[1]:
        
        row.append(i)
    final_predictions.append(row)  


100%|██████████| 79726/79726 [00:00<00:00, 387339.33it/s]


In [64]:
# final_predictions = np.array(final_predictions)

In [67]:
final_predictions

[['img', 'c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'],
 ['img_1.jpg', 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
 ['img_10.jpg', 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
 ['img_100.jpg',
  0.37845856,
  0.00719067,
  0.0022751372,
  0.001494422,
  0.002156919,
  0.0052211233,
  0.00034355544,
  0.0032080617,
  0.012224589,
  0.5874269],
 ['img_1000.jpg',
  4.9716018e-08,
  1.0166412e-07,
  0.07312438,
  1.0725318e-08,
  2.9997052e-06,
  1.7077564e-08,
  0.00025202014,
  6.3466496e-07,
  0.9266195,
  3.2977275e-07],
 ['img_100000.jpg',
  6.383571e-17,
  1.3401694e-24,
  2.0689433e-17,
  1.2969093e-20,
  4.6289557e-07,
  0.9999995,
  8.7527827e-16,
  3.5538367e-23,
  2.2189825e-10,
  3.4681343e-13],
 ['img_100001.jpg',
  6.3886105e-27,
  7.736168e-37,
  0.0,
  1.0,
  2.2705946e-15,
  7.642923e-36,
  0.0,
  0.0,
  5.390376e-32,
  4.7767183e-33],
 ['img_100002.jpg',
  0.0003366595,
  0.0008069177,
  0.00068821455,
  2.3154305e-05,
  0.0004238633,
  5.740564

In [71]:
np.savetxt("ans.csv" , final_predictions, delimiter=",",fmt='% s')